In [33]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

from sklearn.linear_model import LogisticRegression


### https://www.kaggle.com/datasets/andrewmvd/fetal-health-classification

In [24]:
data = pd.read_csv('/kaggle/input/fetal-health-classification/fetal_health.csv')

In [25]:
data

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.000,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.000,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.000,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,140.0,0.000,0.000,0.007,0.000,0.0,0.0,79.0,0.2,25.0,...,137.0,177.0,4.0,0.0,153.0,150.0,152.0,2.0,0.0,2.0
2122,140.0,0.001,0.000,0.007,0.000,0.0,0.0,78.0,0.4,22.0,...,103.0,169.0,6.0,0.0,152.0,148.0,151.0,3.0,1.0,2.0
2123,140.0,0.001,0.000,0.007,0.000,0.0,0.0,79.0,0.4,20.0,...,103.0,170.0,5.0,0.0,153.0,148.0,152.0,4.0,1.0,2.0
2124,140.0,0.001,0.000,0.006,0.000,0.0,0.0,78.0,0.4,27.0,...,103.0,169.0,6.0,0.0,152.0,147.0,151.0,4.0,1.0,2.0


In [26]:
data['fetal_health'] = data['fetal_health'] - 1

In [27]:
y = data['fetal_health']
x = data.drop('fetal_health',axis=1)

In [28]:
x.shape
y.value_counts()
len(y.unique())

3

In [29]:
x_train , x_test , y_train , y_test = train_test_split(x,y,train_size=0.7,random_state=13)

In [30]:
class_weights = dict(
    enumerate(
        compute_class_weight(
            'balanced',
            classes=y.unique(),
            y=y
        )
    
    )

)

In [39]:
inputs=tf.keras.Input(shape =(21,))
x = tf.keras.layers.Dense(128,activation='relu')(inputs)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.Dense(64,activation='relu')(x)
x = tf.keras.layers.Dense(16,activation='relu')(x)

outputs = tf.keras.layers.Dense(3,activation='softmax')(x)

model = tf.keras.Model(inputs=inputs,outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

hist = model.fit(
    x_train,
    y_train,
    validation_split = 0.2,
    batch_size = 25,
    epochs = 100,
    callbacks = tf.keras.callbacks.ReduceLROnPlateau()
)

Epoch 1/100
48/48 [==============================] - 1s 6ms/step - loss: 0.9832 - accuracy: 0.7580 - val_loss: 0.4159 - val_accuracy: 0.8322 - lr: 0.0010
Epoch 2/100
48/48 [==============================] - 0s 3ms/step - loss: 0.4995 - accuracy: 0.8101 - val_loss: 0.3310 - val_accuracy: 0.8792 - lr: 0.0010
Epoch 3/100
48/48 [==============================] - 0s 3ms/step - loss: 0.4270 - accuracy: 0.8328 - val_loss: 0.4673 - val_accuracy: 0.8389 - lr: 0.0010
Epoch 4/100
48/48 [==============================] - 0s 3ms/step - loss: 0.4127 - accuracy: 0.8496 - val_loss: 0.3944 - val_accuracy: 0.8624 - lr: 0.0010
Epoch 5/100
48/48 [==============================] - 0s 3ms/step - loss: 0.4657 - accuracy: 0.8244 - val_loss: 0.4441 - val_accuracy: 0.8255 - lr: 0.0010
Epoch 6/100
48/48 [==============================] - 0s 3ms/step - loss: 0.3904 - accuracy: 0.8395 - val_loss: 0.3908 - val_accuracy: 0.8691 - lr: 0.0010
Epoch 7/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3

In [40]:
model.evaluate(x_test,y_test)

20/20 [==============================] - 0s 2ms/step - loss: 0.2862 - accuracy: 0.9013


[0.28617218136787415, 0.9012539386749268]

In [34]:
model = LogisticRegression()
model.fit(x_train,y_train)
model.score(x_test,y_test)


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.877742946708464